<a href="https://colab.research.google.com/github/peppefdf/CSL_Gipuzkoa/blob/main/DragDrop_Markers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#%pip install osmnx
#%pip install dash
#%pip install dash_leaflet
#%pip install dash_bootstrap_components

import dash
from dash import Dash
import dash_bootstrap_components as dbc
#import dash_html_components as html
from dash import html, callback_context, ALL
from dash import dcc, Output, Input, State, callback
import dash_leaflet as dl
import re
import json


from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

# Custom icon as per official docs https://leafletjs.com/examples/custom-icons/
#    iconUrl='/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/Accessibility_Map/icon_bus_stop.png'
#    iconUrl='/content/drive/MyDrive/Colab Notebooks/CSL_GIPUZKOA/Accessibility_Map/bus_stop_icon.jpg',
#    iconUrl= 'https://raw.githubusercontent.com/pointhi/leaflet-color-markers/master/img/marker-icon-2x-green.png',

custom_icon = dict(
    iconUrl='https://github.com/peppefdf/CSL_Gipuzkoa/blob/ec6ccebbb5fbafa46aba57601273f957717afad6/bus_stop_icon.png',
    iconSize=[38, 95],
    iconAnchor=[22, 94],
    popupAnchor=[-3, -76]
)

"""
# Small example app.
positions = [(57, 11), (57, 10), (58, 10)]
markers = [dl.Marker(dl.Tooltip("After drag and drop, double click on Marker to update its position"), position=pos, id="marker{}".format(i), draggable=True, interactive=True, eventHandlers=True) for i, pos in enumerate(positions)]
Layer_group = dl.LayerGroup(markers, id="layer_group")
"""

app = Dash(prevent_initial_callbacks=True)
app.layout = html.Div([
    html.Div(dl.Map([dl.TileLayer()],
                     center=(57, 10.5), zoom=6, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})),
    html.Div(id='clickdata'),
    html.Button('Update markers visualization', id='update_mark'),
    dcc.Store(id='internal-value', data=[])

])

@app.callback([Output("clickdata", "children"), Output('internal-value','data'),Output('map','children',allow_duplicate=True)],
              [State('internal-value','data')],
              [Input('map','clickData')]
              )
def add_marker(St,clickd):
    marker_lat = clickd['latlng']['lat']
    marker_lon = clickd['latlng']['lng']
    St.append((marker_lat,marker_lon))
    out=''
    for i in range(len(St)):
        out = out + str(St[i][0]) + ', ' + str(St[i][1]) + '; '
    #markers = [dl.Marker(dl.Tooltip("double click on Marker after drag to update its position"), position=pos, id="marker{}".format(i), draggable=True) for i, pos in enumerate(St)]
    #Layer_group = dl.LayerGroup(markers, id="markers_group")
    #Layer_group = dl.LayerGroup(markers, id={"type": "markers_group", "index": 0})
    #markers = [dl.Marker(id={'type': 'marker', 'index': key}, position=data[key]) for key in data]
    markers = [dl.Marker(dl.Tooltip("Click on Marker to save its new position"), position=pos, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=(57, 10.5), zoom=6, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [out,St,newMap]

@app.callback([Output("clickdata", "children",allow_duplicate=True),Output('internal-value','data',allow_duplicate=True),Output('map','children',allow_duplicate=True)],
              [State('internal-value','data')],
              [Input({"type": "marker", "index": ALL},"n_clicks")],
              prevent_initial_callbacks=True)
def remove_marker(St,*args):
    marker_id = callback_context.triggered[0]["prop_id"].split(".")[0].split(":")[1].split(",")[0]
    n_clicks = callback_context.triggered[0]["value"]
    if n_clicks ==2:
       del St[int(marker_id)]
    markers = [dl.Marker(dl.Tooltip("Click on Marker to save its new position"), position=pos, id={'type': 'marker', 'index': i}) for i, pos in enumerate(St)]
    newMap = dl.Map([dl.TileLayer()] + markers,
                     center=(57, 10.5), zoom=6, id="map",
                     style={'width': '100%', 'height': '50vh', 'margin': "auto", "display": "block"})
    return [St,St,newMap]


if __name__ == '__main__':
    app.run_server()

Mounted at /content/drive


<IPython.core.display.Javascript object>